In [26]:
from opensoundscape.preprocess.preprocessors import BasePreprocessor, AudioToSpectrogramPreprocessor, CnnPreprocessor
import torch
import pandas as pd
from pathlib import Path
import numpy as np
import pandas as pd
from opensoundscape.torch.models import cnn
from opensoundscape.torch.architectures import cnn_architectures

In [46]:
df = pd.read_csv('Labeling System xeno-canto Dataset/MDD_Xeno_Canto_DSC180_Labels_uniform_3s.csv')
df['IN FILE'] = ['./Labeling System xeno-canto Dataset/Mixed_Bird/' + f for f in df['IN FILE']]
df['MANUAL ID'].str.contains('Southern Mealy Amazon').value_counts()

False    22018
True         7
Name: MANUAL ID, dtype: int64

In [43]:
df

,IN FILE,CLIP LENGTH,OFFSET,DURATION,SAMPLE RATE,MANUAL ID
0,./Labeling System xeno-canto Dataset/Mixed_Bir...,24.6335,0.0,3.0,44100,Aburria aburri Wattled Guan
1,./Labeling System xeno-canto Dataset/Mixed_Bir...,24.6335,3.0,3.0,44100,Aburria aburri Wattled Guan
2,./Labeling System xeno-canto Dataset/Mixed_Bir...,24.6335,6.0,3.0,44100,Aburria aburri Wattled Guan
3,./Labeling System xeno-canto Dataset/Mixed_Bir...,24.6335,18.0,3.0,44100,Aburria aburri Wattled Guan
4,./Labeling System xeno-canto Dataset/Mixed_Bir...,24.6335,21.0,3.0,44100,Aburria aburri Wattled Guan
...,...,...,...,...,...,...
22020,./Labeling System xeno-canto Dataset/Mixed_Bir...,18.2596,0.0,3.0,44100,Zonotrichia capensis Rufous-collared Sparrow
22021,./Labeling System xeno-canto Dataset/Mixed_Bir...,18.2596,3.0,3.0,44100,Zonotrichia capensis Rufous-collared Sparrow
22022,./Labeling System xeno-canto Dataset/Mixed_Bir...,18.2596,6.0,3.0,44100,Zonotrichia capensis Rufous-collared Sparrow
22023,./Labeling System xeno-canto Dataset/Mixed_Bir...,18.2596,12.0,3.0,44100,Zonotrichia capensis Rufous-collared Sparrow


In [59]:
ef = df.groupby('IN FILE', group_keys=False).apply(lambda ff: ff.sample(1))
ef = ef[ef['MANUAL ID'].str.contains('Attila')]
ef.reset_index(drop=True, inplace=True)

In [60]:
ef

,IN FILE,CLIP LENGTH,OFFSET,DURATION,SAMPLE RATE,MANUAL ID
0,./Labeling System xeno-canto Dataset/Mixed_Bir...,31.03350,0.0,3.0,44100,Attila bolivianus White-eyed Attila
1,./Labeling System xeno-canto Dataset/Mixed_Bir...,105.55200,63.0,3.0,48000,Attila bolivianus White-eyed Attila
2,./Labeling System xeno-canto Dataset/Mixed_Bir...,14.52000,0.0,3.0,48000,Attila bolivianus White-eyed Attila
3,./Labeling System xeno-canto Dataset/Mixed_Bir...,46.96820,39.0,3.0,44100,Attila cinnamomeus Cinnamon Attila
4,./Labeling System xeno-canto Dataset/Mixed_Bir...,54.46530,45.0,3.0,44100,Attila cinnamomeus Cinnamon Attila
5,./Labeling System xeno-canto Dataset/Mixed_Bir...,99.52650,93.0,3.0,44100,Attila cinnamomeus Cinnamon Attila
6,./Labeling System xeno-canto Dataset/Mixed_Bir...,13.21800,0.0,3.0,44100,Attila cinnamomeus Cinnamon Attila
7,./Labeling System xeno-canto Dataset/Mixed_Bir...,97.59350,6.0,3.0,44100,Attila spadiceus Bright-rumped Attila
8,./Labeling System xeno-canto Dataset/Mixed_Bir...,9.27347,6.0,3.0,44100,Attila spadiceus Bright-rumped Attila


In [61]:
from opensoundscape.audio import Audio
clip_duration = 3
clip_overlap = 0
final_clip = None
clip_dir = './temp_clips'
# classes = labels
min_label_overlap = 0.1

ef_split_save = pd.DataFrame(columns=['file', 'start_time', 'end_time', 'SAMPLING RATE', 'MANUAL ID'])

cnt = 0

for i, row in ef.iterrows():
    try:
        audio = Audio.from_file(row['IN FILE'])
    except Exception:
        print(row['IN FILE'] + ' not found')
        continue
    file_name = row['IN FILE'].split('.')
    # print(file_name)
    clip_df = audio.split_and_save(
        clip_dir,
        prefix='..' + file_name[1],
        clip_duration=clip_duration,
        clip_overlap=clip_overlap,
        final_clip='remainder',
        dry_run=False
    )
    clip_df['SAMPLING RATE'] = [row['SAMPLE RATE']] * clip_df.shape[0]
    clip_df['MANUAL ID'] = [row['MANUAL ID']] * clip_df.shape[0]
    clip_df.reset_index(inplace=True)
    # print(clip_df.shape)
    ef_split_save = ef_split_save.append(clip_df, ignore_index=True)
    # print(all_labels.shape)
    # print('*******')
print(ef_split_save)
ef_split_save.to_csv('attila_test.csv')

/var/folders/km/7_v861mx0cb9fp42nvllkmt00000gn/T/ipykernel_36970/2713810586.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ef_split_save = ef_split_save.append(clip_df, ignore_index=True)
/var/folders/km/7_v861mx0cb9fp42nvllkmt00000gn/T/ipykernel_36970/2713810586.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ef_split_save = ef_split_save.append(clip_df, ignore_index=True)
/var/folders/km/7_v861mx0cb9fp42nvllkmt00000gn/T/ipykernel_36970/2713810586.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ef_split_save = ef_split_save.append(clip_df, ignore_index=True)
/var/folders/km/7_v861mx0cb9fp42nvllkmt00000gn/T/ipykernel_36970/2713810586.py:33: FutureWarning: The frame.append method is deprecated and will be rem

                                                  file start_time   end_time  \
0    ./temp_clips/../Labeling System xeno-canto Dat...        0.0        3.0   
1    ./temp_clips/../Labeling System xeno-canto Dat...        3.0        6.0   
2    ./temp_clips/../Labeling System xeno-canto Dat...        6.0        9.0   
3    ./temp_clips/../Labeling System xeno-canto Dat...        9.0       12.0   
4    ./temp_clips/../Labeling System xeno-canto Dat...       12.0       15.0   
..                                                 ...        ...        ...   
158  ./temp_clips/../Labeling System xeno-canto Dat...       96.0  97.593469   
159  ./temp_clips/../Labeling System xeno-canto Dat...        0.0        3.0   
160  ./temp_clips/../Labeling System xeno-canto Dat...        3.0        6.0   
161  ./temp_clips/../Labeling System xeno-canto Dat...        6.0        9.0   
162  ./temp_clips/../Labeling System xeno-canto Dat...        9.0   9.273469   

    SAMPLING RATE                      

/var/folders/km/7_v861mx0cb9fp42nvllkmt00000gn/T/ipykernel_36970/2713810586.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ef_split_save = ef_split_save.append(clip_df, ignore_index=True)
/var/folders/km/7_v861mx0cb9fp42nvllkmt00000gn/T/ipykernel_36970/2713810586.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ef_split_save = ef_split_save.append(clip_df, ignore_index=True)


## Investigate df

In [1]:
import pandas as pd
df = pd.read_csv('screaming_pyha_test.csv')
ef = pd.read_csv('flycatcher_test.csv')
ff = pd.read_csv('antbird_test.csv')

In [3]:
gf = df.append(ef, ignore_index=True)
hf = gf.append(ff, ignore_index=True)
hf.shape

/var/folders/km/7_v861mx0cb9fp42nvllkmt00000gn/T/ipykernel_36970/7459103.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gf = df.append(ef, ignore_index=True)
/var/folders/km/7_v861mx0cb9fp42nvllkmt00000gn/T/ipykernel_36970/7459103.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hf = gf.append(ff, ignore_index=True)


(2653, 6)

In [8]:
hf.loc[hf['MANUAL ID'].str.contains('Flycatcher'), 'MANUAL ID'] = 'Flycatcher'
hf.loc[hf['MANUAL ID'].str.contains('Antbird'), 'MANUAL ID'] = 'Antbird'
hf['MANUAL ID'] = hf['MANUAL ID'].map({'bird': 'Piha', 'Flycatcher': 'Flycatcher', 'Antbird': 'Antbird'})
hf['MANUAL ID'].value_counts()

Flycatcher    1348
Antbird       1106
Piha           199
Name: MANUAL ID, dtype: int64

In [21]:
from sklearn.model_selection import train_test_split
train_df, valid_df = train_test_split(hf, test_size=0.05, random_state=42)
train_df, test_df = train_test_split(train_df, test_size=0.1, random_state=42)

In [23]:
train_df['MANUAL ID'].value_counts()

Flycatcher    1162
Antbird        931
Piha           175
Name: MANUAL ID, dtype: int64

In [24]:
valid_df['MANUAL ID'].value_counts()

Antbird       62
Flycatcher    62
Piha           9
Name: MANUAL ID, dtype: int64

In [25]:
test_df['MANUAL ID'].value_counts()

Flycatcher    124
Antbird       113
Piha           15
Name: MANUAL ID, dtype: int64